# images

In [ ]:
import glob

In [ ]:
images = glob.glob("/root/data/lice-data/lice_only/*/*/*/*/*.jpg")

In [ ]:
print("total number of images: {}".format(len(images)))

# generator

In [ ]:
import keras

In [ ]:
import glob
import json
import os

import cv2
import numpy as np
import skimage.io as io
from albumentations import PadIfNeeded, HorizontalFlip, VerticalFlip, Compose, RandomRotate90
from keras.callbacks import Callback


CLASS_MAP = {"ADULT_FEMALE": 0,
             "FIXED_MOVING": 1,
             "UNSURE": 2}

In [ ]:
def get_data_dict(data):
    """quick hack for uniform sampling during training"""
    classdict = {}
    for d in data:
        liceclass = d.split('/')[-2]
        if liceclass == "UNSURE":
            continue
        if liceclass == "MOVING" or liceclass == "FIXED":
            liceclass = "FIXED_MOVING"
        if liceclass not in classdict:
            classdict[liceclass] = []
        classdict[liceclass].append(d)
    return classdict

# load model

In [ ]:
import os

In [ ]:
!nvidia-smi

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import random
from collections import Counter

In [ ]:
batch_size = 32
input_shape = [64, 64, 3]
random.shuffle(images)
cutoff = int(0.8*len(images))

In [ ]:
#     aug = Compose([PadIfNeeded(p=1.0, min_height=input_shape[0], min_width=input_shape[1], border_mode=0),
#                    HorizontalFlip(p=0.0),
#                    RandomRotate90(p=0.0)])

In [ ]:
# img_path = train_dict["ADULT_FEMALE"][10]
# image = io.imread(img_path)
# image = RandomRotate90(p=1.0)(image=image)["image"]
# print(image.shape)
# plt.imshow(image)
# plt.show()
# height, width, _ = image.shape
# print(height, width)
# if height > width:
#     print("height")
#     ratio = input_shape[0] / float(height)
#     print(ratio)
#     x = cv2.resize(image, (int(width*ratio), input_shape[1]), interpolation=cv2.INTER_LINEAR)
#     print(x.shape)
# else:
#     print("width")
#     ratio = input_shape[1] / float(width)
#     print(ratio)
#     x = cv2.resize(image, (input_shape[0], int(height*ratio)), interpolation=cv2.INTER_LINEAR)
#     print(x.shape)
# plt.imshow(x)
# plt.show()   

# augmented = aug(image=x)
# x = augmented["image"]

# plt.imshow(x)
# plt.show()

In [ ]:
def generator(classdict, batch_size, input_shape):
    """data generator"""
#     aug = Compose([PadIfNeeded(p=1, min_height=input_shape[0], min_width=input_shape[1], border_mode=0)])
    aug = Compose([PadIfNeeded(p=1.0, min_height=input_shape[0], min_width=input_shape[1], border_mode=0),
                   HorizontalFlip(p=0.5),
                   RandomRotate90(p=0.0)])
#     aug = Compose([HorizontalFlip(p=0.5)])
    classes = len(list(classdict.keys()))
    while True:
        x_batch = np.zeros((batch_size, input_shape[0], input_shape[1], input_shape[2]), dtype=np.uint8)
        y_batch = np.empty((batch_size, classes))
        for i in range(batch_size):
            liceclass = np.random.choice(list(classdict.keys()))
            img_path = np.random.choice(classdict[liceclass])
            image = io.imread(img_path)
            height, width, _ = image.shape
            if height > width:
                ratio = input_shape[0] / float(height)
                x = cv2.resize(image, (int(width*ratio), input_shape[1]), interpolation=cv2.INTER_LINEAR)
            else:
                ratio = input_shape[1] / float(width)
                x = cv2.resize(image, (input_shape[0], int(height*ratio)), interpolation=cv2.INTER_LINEAR)
            augmented = aug(image=x)
            x = augmented["image"]
            # x = cv2.resize(augmented["image"], (input_shape[0], input_shape[1]), interpolation=cv2.INTER_LINEAR)
            y = np.zeros(classes)
            y[CLASS_MAP[liceclass]] = 1
            x_batch[i, ...] = x
            y_batch[i, ...] = y
        yield x_batch, y_batch

In [ ]:
train_data = images[:cutoff]
steps_per_epoch = len(train_data) // batch_size
train_dict = get_data_dict(train_data)
train_generator = generator(train_dict, batch_size, input_shape)
# config["train_dataset_size"] = len(train_data)
print("Train dataset size: {}. Steps per epoch: {}".format(len(train_data), steps_per_epoch))

# validation data
validation_data = images[cutoff:]
val_steps_per_epoch = len(validation_data) // batch_size
validation_dict = get_data_dict(validation_data)
validation_generator = generator(validation_dict, batch_size, input_shape)
# config["val_dataset_size"] = len(validation_data)
print("Val dataset size: {}. Steps per epoch: {}".format(len(validation_data), val_steps_per_epoch))

In [ ]:
Counter([p.split('/')[-2] for p in images]).most_common()

In [ ]:
Counter([p.split('/')[-2] for p in train_data])

In [ ]:
Counter([p.split('/')[-2] for p in validation_data])

# PLOT

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X, Y = next(validation_generator)

In [ ]:
# for p in train_dict["ADULT_FEMALE"]:
#     plt.imshow(io.imread(p))
#     plt.show()

In [ ]:
# for i in range(batch_size):
#     plt.imshow(X[i, ...].squeeze())
#     plt.show()

# TRAIN

In [ ]:
from keras.applications.resnet50 import ResNet50
# from keras.applications.mobilenet import _depthwise_conv_block
from utils import depthwise_conv_block, conv_block
from keras import layers
from keras.models import Model
from keras.callbacks import  ModelCheckpoint
from keras import Sequential
from keras.optimizers import Adam

In [ ]:
resnet50 = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
x = layers.GlobalAveragePooling2D(name='avg_pool')(resnet50.output)
x = layers.Dense(2, activation='softmax', name='fc1000')(x)
model = Model(inputs=[resnet50.input], outputs=[x])

In [ ]:
# img_input = layers.Input(shape=input_shape)
# # Block 1
# x = layers.Conv2D(64, (3, 3),
#       activation='relu',
#       padding='same',
#       name='block1_conv1')(img_input)
# x = layers.Conv2D(64, (3, 3),
#       activation='relu',
#       padding='same',
#       name='block1_conv2')(x)
# x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# # Block 2
# x = layers.Conv2D(128, (3, 3),
#       activation='relu',
#       padding='same',
#       name='block2_conv1')(x)
# x = layers.Conv2D(128, (3, 3),
#       activation='relu',
#       padding='same',
#       name='block2_conv2')(x)
# x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# x = layers.Flatten(name='flatten')(x)
# x = layers.Dense(4096, activation='relu', name='fc1')(x)
# x = layers.Dense(3, activation='softmax', name='predictions')(x)

In [ ]:
classes = 2
alpha = 0.5
dropout = 1e-3
depth_multiplier = 1
shape = (1, 1, int(512 * alpha))

In [ ]:
img_input = layers.Input(shape=input_shape)
x = conv_block(img_input, 32, alpha, strides=(2, 2))
x = depthwise_conv_block(x, 64, alpha, depth_multiplier, block_id=1)

x = depthwise_conv_block(x, 128, alpha, depth_multiplier,
                          strides=(2, 2), block_id=2)
x = depthwise_conv_block(x, 128, alpha, depth_multiplier, block_id=3)

x = depthwise_conv_block(x, 256, alpha, depth_multiplier,
                          strides=(2, 2), block_id=4)
x = depthwise_conv_block(x, 256, alpha, depth_multiplier, block_id=5)

x = depthwise_conv_block(x, 512, alpha, depth_multiplier,
                          strides=(2, 2), block_id=6)
x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=7)
x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=8)
x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=9)
x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=10)
x = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=11)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Reshape(shape, name='reshape_1')(x)
x = layers.Dropout(dropout, name='dropout')(x)
x = layers.Conv2D(classes, (1, 1),
                  padding='same',
                  name='conv_preds')(x)
x = layers.Activation('softmax', name='act_softmax')(x)
x = layers.Reshape((classes,), name='reshape_2')(x)
model = Model(inputs=[img_input], outputs=[x])

In [ ]:
adam = Adam(lr=1e-3)
model.compile(adam, loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch // 10,
                              epochs=19,
                              verbose=1,
                              validation_data=validation_generator,
                              validation_steps=val_steps_per_epoch // 10)

In [ ]:
h = history.history

In [ ]:
plt.plot(h["loss"])
plt.plot(h["val_loss"])
plt.legend(["loss", "val_loss"])
plt.ylim([0, 2])
plt.show()

In [ ]:
plt.plot(h["categorical_accuracy"])
plt.plot(h["val_categorical_accuracy"])
plt.legend(["acc", "val_acc"])
plt.show()

# confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_true_all = []
y_pred_all = []
for i in range(10):
    X, Y_true = next(validation_generator)
    Y_pred = model.predict_on_batch(X)
    for j in range(batch_size):
        y_true_all.append(np.argmax(Y_true[j, :]))
        y_pred_all.append(np.argmax(Y_pred[j, :]))

In [ ]:
CLASS_MAP = {"ADULT_FEMALE": 2,
             "FIXED_MOVING": 1,
             "UNSURE": 0}

In [ ]:
confusion_matrix(y_true_all, y_pred_all)

In [ ]:
(152+130) / (152+130+11+27)